In [1]:
# load necessary files
import json
import pandas as pd
import yfinance as yf

#etfs = pd.read_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/23-24 Competition ETF List-FINAL.xlsx')
stock = pd.read_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/all_data.xlsx', index_col=0)
BSE_all = pd.read_csv('/Users/tprazak/Documents/Wharton_Investment_Competition/Tabulky/BSE_all.csv', index_col=False)
equity_etfs = pd.read_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/equity_etfs.xlsx')

In [7]:
stock.loc[stock['GICS Sector'] == 'Information Technology']

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
11,Cielo SA,CIEL3.SA,B3 S.A.,Information Technology,Software and Services,IT Services,Data Processing and Outsourced Services,South America,BRL,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Astra Microwave Products Ltd,ASTRAMICRO.BO,BSE LTD,Information Technology,Technology Hardware and Equipment,Communications Equipment,Communications Equipment,Asia-Pacific,INR,584.950,...,91 40 4661 8048,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,eClerx Services Ltd,ECLERX.BO,BSE LTD,Information Technology,Software and Services,IT Services,Data Processing and Outsourced Services,Asia-Pacific,INR,2748.950,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,HCL Technologies Ltd,HCLTECH.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1348.450,...,91 12 0468 0330,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Infosys Ltd,INFY.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1457.650,...,91 80 2852 0362,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,LTIMindtree,LTIM.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,5582.000,...,91 22 4313 0997,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NIIT Ltd,NIITLTD.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,114.000,...,91 12 4429 3333,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Tata Consultancy Services Ltd,TCS.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,3495.100,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Tech Mahindra Ltd,TECHM.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1221.050,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Wipro Ltd,WIPRO.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,414.000,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
stock.loc[stock['GICS Industry Group'] == 'Semiconductors & Semiconductor Equipment']['returnOnEquity'].mean()

0.17756416500000002

In [3]:
gendered_execs = json.loads(open('/Users/tprazak/Documents/Wharton_Investment_Competition/gendered_execs.json').read())

In [ ]:
# changes to tickers to make them yahoo finance compatible
exchange_regions = {
    "B3 S.A.":("Brazil", "South America", ".SA"),
    "BMV":("Mexico", "South America", ".MX"),
    "BSE LTD":("India", "Asia-Pacific", ".BO"),
    "Euronext":("Netherlands", "Europe", ".AS"),
    "Frankfurt Stock Exchange":("Germany", "Europe", ".F"),
    "Hong Kong Exchanges And Clearing Ltd":("Hong Kong", "Asia-Pacific", ".HK"),
    "Korea Exchange":("Korea", "Asia-Pacific", ".KS"),
    "London Stock Exchange":("UK", "Europe", ".L"),
    "Nasdaq":("US", "North America", ""),
    "New York Stock Exchange, Inc.":("US", "North America", ""),
    "Shanghai Stock Exchange":("China", "Asia-Pacific", ".SS"),
    "Shenzhen Stock Exchange":("China", "Asia-Pacific", ".SZ"),
    "SIX Swiss Exchange":("Switzerland", "Europe", ".SW"),
    "Toronto Stock Exchange":("Canada", "North America", ".TO"),
}

stock['Country'] = stock['Exchange'].map(lambda x: exchange_regions[x][0])
stock['Region'] = stock['Exchange'].map(lambda x: exchange_regions[x][1])
for i in stock.index:
    stock.loc[i, 'Ticker'] = 'RBC' if stock.loc[i, 'Ticker'] == 'ROLL' else stock.loc[i, 'Ticker']
     
    if stock.loc[i, 'Exchange'] == 'Hong Kong Exchanges And Clearing Ltd':
        if len(stock.loc[i, 'Ticker']) > 4:
            stock.loc[i, 'Ticker'] = str(stock.loc[i, 'Ticker'])[len(stock.loc[i, 'Ticker'])-4:]

    if stock.loc[i, 'Exchange'] == 'Korea Exchange':
        if len(str(stock.loc[i, 'Ticker'])) < 6:
            stock.loc[i, 'Ticker'] = '0'*(6-len(str(stock.loc[i, 'Ticker']))) + str(stock.loc[i, 'Ticker'])

    if stock.loc[i, 'Exchange'] in ['London Stock Exchange', 'Toronto Stock Exchange']:
        if '.' in stock.loc[i, 'Ticker']:
            stock.loc[i, 'Ticker'] = stock.loc[i, 'Ticker'].replace('.', '-')

    if stock.loc[i, 'Exchange'] == 'BSE LTD':
        ticker = int(stock.loc[i, 'Ticker'])
        bse_ticker = BSE_all.loc[BSE_all['Security Code'] == ticker].iloc[0]
        first = bse_ticker.at['Security Id']
        stock.loc[i, 'Ticker'] = first
    
    if stock.loc[i, 'Ticker'] in ['BN', 'OR', 'MC']:
        stock.loc[i, 'Ticker'] = str(stock.loc[i, 'Ticker']) + '.PA'
    elif stock.loc[i, 'Ticker'] in ['UCB']:
        stock.loc[i, 'Ticker'] = str(stock.loc[i, 'Ticker']) + '.BR'
    else:
        stock.loc[i, 'Ticker'] = str(stock.loc[i, 'Ticker']) + exchange_regions[stock.loc[i, 'Exchange']][2]


In [3]:
exclude_columns = ['sharesOutstanding',
                   'phone',
                   'regularMarketVolume',
                   'targetMedianPrice',
                   'fiveYearAvgDividendYield',
                   'previousClose',
                   'totalRevenue',
                   '52WeekChange',
                   'address2',
                   'industrySymbol',
                   'boardRisk',
                   'grossMargins',
                   'fax',
                   'targetLowPrice',
                   'sectorKey',
                   'overallRisk',
                   'exDividendDate',
                   'dateShortInterest',
                   'compensationAsOfEpochDate',
                   'revenuePerShare',
                   'pegRatio',
                   'recommendationKey',
                   'regularMarketDayHigh',
                   'gmtOffSetMilliseconds',
                   'priceHint',
                   'industryDisp',
                   'firstTradeDateEpochUtc',
                   'sharesShortPreviousMonthDate',
                   'shareHolderRightsRisk',
                   'netIncomeToCommon',
                   'totalDebt',
                   'operatingMargins',
                   'forwardEps',
                   'industryKey',
                   'dayHigh',
                   'totalCash',
                   'targetHighPrice',
                   'numberOfAnalystOpinions',
                   'debtToEquity',
                   'fullTimeEmployees',
                   'symbol',
                   'maxAge',
                   'twoHundredDayAverage',
                   'enterpriseToRevenue',
                   'regularMarketPreviousClose',
                   'financialCurrency',
                   'returnOnAssets',
                   'address1',
                   'regularMarketDayLow',
                   'priceToSalesTrailing12Months',
                   'sectorDisp',
                   'lastSplitDate',
                   'compensationRisk',
                   'averageDailyVolume10Day',
                   'grossProfits',
                   'targetMeanPrice',
                   'lastFiscalYearEnd',
                   'fiftyDayAverage',
                   'zip',
                   'quickRatio',
                   'uuid',
                   'nextFiscalYearEnd',
                   'exchange',
                   'currentRatio',
                   'messageBoardId',
                   'fiftyTwoWeekHigh',
                   'bidSize',
                   'ask',
                   'askSize',
                   'dayLow',
                   'revenueGrowth',
                   'forwardPE',
                   'averageVolume10days',
                   'shortPercentOfFloat',
                   'quoteType',
                   'governanceEpochDate',
                   'city',
                   'fiftyTwoWeekLow',
                   'sector',
                   'sharesShortPriorMonth',
                   'shortName',
                   'lastSplitFactor',
                   'currentPrice',
                   'shortRatio',
                   'underlyingSymbol',
                   'mostRecentQuarter',
                   'trailingPegRatio',
                   'lastDividendDate',
                   'floatShares',
                   'SandP52WeekChange',
                   'sharesShort',
                   'sharesPercentSharesOut',
                   'bid',
                   'recommendationMean',
                   'state',
                   'averageVolume',
                   'auditRisk',
                   'dividendRate',
                   'volume',
                   'trailingEps',
                   'timeZoneFullName',
                   'industry',
                   'longName',
                   'profitMargins',
                   'regularMarketOpen',
                   'totalCashPerShare',
                   'lastDividendValue',
                   'impliedSharesOutstanding',
                   'timeZoneShortName']

In [4]:
# load and save data from yahoo finance
present_keys = stock.columns.tolist()
for i in stock.index:
    ticker = stock.loc[i, 'Ticker']
    try:
        info = yf.Ticker(ticker).info
    except:
        print(ticker)
        continue
    keys = info.keys()
    for j in keys:
        if j not in present_keys:
            stock[j] = None if j != 'companyOfficers' else []
            present_keys.append(j)
        if info[j] == 'Infinity':
            info[j] = 0
        stock.at[i, j] = info[j]
stock['bookValue'] *= 10**9
stock.to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/all_data.xlsx')
total_stock_data = stock.copy(deep=True)
stock.drop(columns=exclude_columns, inplace=True)

In [10]:
# load and save data from yahoo finance manually
info = yf.Ticker('UL').info
keys = info.keys()
for j in keys:
    if j not in present_keys:
        total_stock_data[j] = None if j != 'companyOfficers' else []
        present_keys.append(j)
    if info[j] == 'Infinity':
        info[j] = 0
    total_stock_data.at[i, j] = info[j]

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/UL?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true

In [12]:
yf.Ticker('AAPL').info.__len__()

132

In [8]:
stock = total_stock_data.copy(deep=True)
total_stock_data.to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/all_data.xlsx')
stock.drop(columns=exclude_columns, inplace=True)

NameError: name 'total_stock_data' is not defined

In [10]:
# calculate women in management ratio
no_execs_found = set()
no_gender_identified = []
stock['womenInMgmtRatio'] = None

for i in stock.index:
    ticker = stock.loc[i, 'Ticker']
    execs = stock.loc[i, 'companyOfficers']
    try:
        total_company_execs = len(execs)
    except TypeError:
        no_execs_found.add(ticker)
        continue
    women_execs = 0
    for officer in execs:
        if officer['name'].startswith('Mrs') or officer['name'].startswith('Ms'):
            women_execs += 1
        elif officer['name'].startswith('Mr'):
            pass
        else:
            if ticker in gendered_execs.keys():
                if officer['name'] in gendered_execs[ticker].keys():
                    officer_gender = gendered_execs[ticker][officer['name']]
                    if officer_gender == 'F':
                        women_execs += 1
                else:
                    no_gender_identified.append((ticker, officer['name']))
    women_mgmt_ratio = women_execs / total_company_execs if total_company_execs != 0 else 0
    stock.at[i, 'womenInMgmtRatio'] = women_mgmt_ratio

print("These are executives of unidentifiable gender: ")
print(no_gender_identified)

These are executives of unidentifiable gender: 
[('PFE', 'Dr. Albert  Bourla D.V.M., Ph.D.')]


In [13]:
gendered_execs['PFE']['Dr. Albert  Bourla D.V.M., Ph.D.'] = 'M'

In [3]:
stock.loc[stock['GICS Sector'] == 'Information Technology']

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
11,Cielo SA,CIEL3.SA,B3 S.A.,Information Technology,Software and Services,IT Services,Data Processing and Outsourced Services,South America,BRL,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Astra Microwave Products Ltd,ASTRAMICRO.BO,BSE LTD,Information Technology,Technology Hardware and Equipment,Communications Equipment,Communications Equipment,Asia-Pacific,INR,584.950,...,91 40 4661 8048,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,eClerx Services Ltd,ECLERX.BO,BSE LTD,Information Technology,Software and Services,IT Services,Data Processing and Outsourced Services,Asia-Pacific,INR,2748.950,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,HCL Technologies Ltd,HCLTECH.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1348.450,...,91 12 0468 0330,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Infosys Ltd,INFY.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1457.650,...,91 80 2852 0362,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,LTIMindtree,LTIM.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,5582.000,...,91 22 4313 0997,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NIIT Ltd,NIITLTD.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,114.000,...,91 12 4429 3333,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Tata Consultancy Services Ltd,TCS.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,3495.100,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Tech Mahindra Ltd,TECHM.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,1221.050,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Wipro Ltd,WIPRO.BO,BSE LTD,Information Technology,Software and Services,IT Services,IT Consulting and Other Services,Asia-Pacific,INR,414.000,...,NaN,1.703981e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# write gendered execs to json
import json
with open('/Users/tprazak/Documents/Wharton_Investment_Competition/gendered_execs.json', 'w') as f:
    json.dump(gendered_execs, f)

In [8]:
total_stock_data[['Company Name','GICS Sector', 'Ticker', 'dividendYield', 'payoutRatio', 'trailingPE', 'womenInMgmtRatio', 'marketCap', 'bookValue']].sort_values(by='dividendYield', ascending=False)

,Company Name,GICS Sector,Ticker,dividendYield,payoutRatio,trailingPE,womenInMgmtRatio,marketCap,bookValue
97,Vedanta Ltd,Materials,VEDL.BO,0.2965,4.9533,16.720547,0.4,8.645010e+11,85024000000.0
160,Tullow Oil PLC,Energy,TLW.L,0.2309,0.0000,NaN,0.3,5.575991e+08,-157000000.0
13,Companhia Siderurgica Nacional,Materials,CSNA3.SA,0.1944,4.9622,33.956520,0.1,2.071353e+10,13395000000.0
29,Petroleo Brasileiro SA Petrobras,Energy,PETR3.SA,0.1927,0.8902,3.726555,0.1,4.765371e+11,29658000000.0
274,VEON Ltd,Communication Services,VEON,0.1744,0.0000,2.842857,0.1,1.397555e+09,15179000000.0
...,...,...,...,...,...,...,...,...,...
483,Lithium Americas,Materials,LAC,NaN,0.0000,NaN,NaN,1.133140e+09,7792000000.0
503,Air Canada,Industrials,AC.TO,NaN,0.0000,2.975083,0.2,6.420179e+09,1534000000.0
505,Aurora Cannabis Inc,Health Care,ACB.TO,NaN,0.0000,NaN,0.3,3.087825e+08,1198000000.0
510,Canada Goose Holdings Inc,Consumer Discretionary,GOOS.TO,NaN,0.0000,29.260000,0.4,1.476474e+09,3459000000.0


In [11]:
total_stock_data

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
0,Ambev SA,ABEV3.SA,B3 S.A.,Consumer Staples,"Food, Beverage and Tobacco",Beverages,Brewers,South America,BRL,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Atacadao SA,CRFB3.SA,B3 S.A.,Consumer Staples,Food and Staples Retailing,Food and Staples Retailing,Hypermarkets and Super Centers,South America,BRL,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B3 SA - Brasil Bolsa Balcao,B3SA3.SA,B3 S.A.,Financials,Diversified Financials,Capital Markets,Financial Exchanges & Data,South America,BRL,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Banco BTG Pactual S.A.,BPAC11.SA,B3 S.A.,Financials,Diversified Financials,Capital Markets,Diversified Capital Markets,South America,BRL,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Banco do Brasil SA BB Brasil,BBAS3.SA,B3 S.A.,Financials,Banks,Banks,Diversified Banks,South America,BRL,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Open Text Corp,OTEX.TO,Toronto Stock Exchange,Information Technology,Software and Services,Software,Application Software,North America,CAD,54.33,...,519 888 7111,1.703981e+09,2498260.0,2489157.0,1.697155e+09,1.700006e+09,0.0092,4.69,0.0094,NaN
521,Restaurant Brands International Inc,QSR.TO,Toronto Stock Exchange,Consumer Discretionary,Consumer Services,"Hotels, Restaurants and Leisure",Restaurants,North America,CAD,96.27,...,NaN,1.672445e+09,2670862.0,2954733.0,1.697155e+09,1.700006e+09,0.0086,3.84,0.0100,NaN
522,Royal Bank of Canada,RY.TO,Toronto Stock Exchange,Financials,Banks,Commercial Banks,Diversified Banks,North America,CAD,122.60,...,NaN,1.672445e+09,10887777.0,9119443.0,1.697155e+09,1.700006e+09,0.0078,2.01,NaN,NaN
523,Shopify Inc,SHOP.TO,Toronto Stock Exchange,Information Technology,Software and Services,IT Services,Internet Services & Infrastructure,North America,CAD,98.00,...,NaN,1.672445e+09,14176704.0,9370047.0,1.697155e+09,1.700006e+09,0.0110,3.77,0.0197,NaN


In [17]:
non_miso_stocks = stock.loc[stock['womenInMgmtRatio'] > 0]
decentPE = non_miso_stocks.loc[non_miso_stocks['trailingPE'] < 30].loc[non_miso_stocks['trailingPE'] > 0]
existentDiv = decentPE.loc[decentPE['dividendYield'] > 0]
materials = existentDiv.loc[existentDiv['GICS Sector'] == 'Materials']
health_care = existentDiv.loc[existentDiv['GICS Sector'] == 'Health Care']

In [103]:
health_care[['Company Name', 'GICS Industry Group','GICS Industry', 'GICS Sub-Industry', 'Ticker', 'dividendYield', 'payoutRatio', 'trailingPE', 'womenInMgmtRatio', 'marketCap', 'bookValue', 'freeCashflow', 'returnOnEquity', 'longBusinessSummary', 'ebitdaMargins', 'heldPercentInstitutions']].sort_values(by='dividendYield', ascending=False)

,Company Name,GICS Industry Group,GICS Industry,GICS Sub-Industry,Ticker,dividendYield,payoutRatio,trailingPE,womenInMgmtRatio,marketCap,bookValue,freeCashflow,returnOnEquity,longBusinessSummary,ebitdaMargins,heldPercentInstitutions
410,Pfizer Inc,Pharmaceuticals and Biotechnology,Pharmaceuticals,Pharmaceuticals,PFE,0.0501,0.4309,8.710107,0.3,1.849052e+11,1.754100e+10,1.235237e+10,0.23068,"Pfizer Inc. discovers, develops, manufactures,...",0.39864,0.70767
310,Bristol-Myers Squibb Co,Pharmaceuticals and Biotechnology,Pharmaceuticals,Pharmaceuticals,BMY,0.0397,0.5984,15.257978,0.2,1.198517e+11,1.530500e+10,1.264362e+10,0.24662,"Bristol-Myers Squibb Company discovers, develo...",0.41368,0.78206
361,GlaxoSmithKline PLC,Pharmaceuticals and Biotechnology,Pharmaceuticals,Pharmaceuticals,GSK,0.0381,0.4622,11.905844,0.6,7.508219e+10,3.131000e+09,2.112350e+10,0.33219,"GSK plc, together with its subsidiaries, engag...",0.33661,0.13596
501,Roche Holding AG,Pharmaceuticals & Biotechnology,Pharmaceuticals,35202010,ROG.SW,0.0376,0.6934,18.417213,0.4,2.034413e+11,3.320000e+10,1.101537e+10,0.40253,Roche Holding AG engages in the pharmaceutical...,0.36426,0.40458
205,Gilead Sciences Inc,Pharmaceuticals and Biotechnology,Biotechnology,Biotechnology,GILD,0.0373,0.6805,18.501150,0.6,1.002789e+11,1.696700e+10,9.621500e+09,0.26348,"Gilead Sciences, Inc., a biopharmaceutical com...",0.44423,0.86845
405,Novartis AG,Pharmaceuticals and Biotechnology,Pharmaceuticals,Pharmaceuticals,NVS,0.0363,0.9719,26.802778,0.2,2.006538e+11,2.506800e+09,1.191050e+10,0.13315,"Novartis AG researches, develops, manufactures...",0.37740,0.08040
19,Hypera SA,Pharmaceuticals and Biotechnology,Pharmaceuticals,Pharmaceuticals,HYPE3.SA,0.0356,0.4507,12.435898,0.125,2.147860e+10,1.746700e+10,-6.558510e+08,0.16407,Hypera S.A. operates as a pharmaceutical compa...,0.33286,0.38682
178,Amgen Inc,Pharmaceuticals and Biotechnology,Biotechnology,Biotechnology,AMGN,0.0300,0.5485,19.152863,0.1,1.521365e+11,1.267700e+10,1.005412e+10,1.73457,"Amgen Inc. discovers, develops, manufactures, ...",0.51776,0.79325
498,Shenzhen Mindray Bio-Medical Electronics Co Ltd,Health Care Equipment and Services,Health Care Equipment and Supplies,Health Care Equipment,300760.SZ,0.0169,0.5065,29.905510,0.5,3.223393e+11,2.762000e+10,6.247677e+09,0.35458,"Shenzhen Mindray Bio-Medical Electronics Co., ...",0.35858,0.20229
465,UnitedHealth Group Inc,Health Care Equipment and Services,Health Care Providers and Services,Managed Health Care,UNH,0.0132,0.3057,24.043460,0.1,4.971016e+11,9.568800e+10,2.302550e+10,0.25299,UnitedHealth Group Incorporated operates as a ...,0.09595,0.88991


In [90]:
drop_materials = [97, 13, 108, 464, 66, 107, 353, 284, 140]

In [92]:
for i in drop_materials:
    materials.drop(i, inplace=True)

/var/folders/y0/sj5bvt0j4qvf3hsh01p94f8h0000gn/T/ipykernel_1611/882205342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materials.drop(i, inplace=True)
/var/folders/y0/sj5bvt0j4qvf3hsh01p94f8h0000gn/T/ipykernel_1611/882205342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materials.drop(i, inplace=True)
/var/folders/y0/sj5bvt0j4qvf3hsh01p94f8h0000gn/T/ipykernel_1611/882205342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [20]:
existentDiv.loc[existentDiv['GICS Sector'] == 'Materials'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/materials.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Communication Services'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/communication_services.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Consumer Discretionary'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/consumer_discretionary.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Consumer Staples'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/consumer_staples.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Energy'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/energy.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Financials'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/financials.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Health Care'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/health_care.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Industrials'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/industrials.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Information Technology'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/information_technology.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Real Estate'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/real_estate.xlsx')
existentDiv.loc[existentDiv['GICS Sector'] == 'Utilities'].to_excel('/Users/tprazak/Documents/Wharton_Investment_Competition/utilities.xlsx')

In [263]:
print(existentDiv.loc[existentDiv['Ticker'] == 'BHP.L']['longBusinessSummary'].iloc[0])

BHP Group Limited operates as a resources company in Australia, Europe, China, Japan, India, South Korea, the rest of Asia, North America, South America, and internationally. The company operates through Copper, Iron Ore, and Coal segments. It engages in the mining of copper, silver, zinc, molybdenum, uranium, gold, iron ore, and metallurgical and energy coal. The company is also involved in mining, smelting, and refining of nickel; and potash development activities. In addition, it provides towing, freight, marketing and trading, marketing support, finance, administrative, and other services. BHP Group Limited was founded in 1851 and is headquartered in Melbourne, Australia.


In [101]:
existentDiv.loc[existentDiv['GICS Sector'] == 'Utilities'][['Company Name', 'GICS Industry Group','GICS Industry', 'GICS Sub-Industry', 'Ticker', 'dividendYield', 'payoutRatio','trailingPE', 'womenInMgmtRatio', 'marketCap', 'bookValue', 'freeCashflow', 'returnOnEquity', 'longBusinessSummary', 'ebitdaMargins']].sort_values(by='dividendYield', ascending=False)

,Company Name,GICS Industry Group,GICS Industry,GICS Sub-Industry,Ticker,dividendYield,payoutRatio,trailingPE,womenInMgmtRatio,marketCap,bookValue,freeCashflow,returnOnEquity,longBusinessSummary,ebitdaMargins
15,CPFL Energia S.A.,Utilities,Electric Utilities,Electric Utilities,CPFE3.SA,0.1610,0.4371,6.796258,0.1,3.766705e+10,1.547500e+10,3.918723e+09,0.34123,"CPFL Energia S.A. generates, transmits, distri...",0.32859
17,ENGIE Brasil Energia SA,Utilities,Independent Power and Renewable Electricity Pr...,Renewable Electricity,EGIE3.SA,0.0869,0.8355,10.256927,0.1,3.322459e+10,1.022400e+10,2.544702e+09,0.37619,"Engie Brasil Energia S.A., together with its s...",0.53663
110,China Gas Holdings Ltd,Utilities,Gas Utilities,Gas Utilities,0384.HK,0.0662,0.6875,9.325000,0.444444,4.054936e+10,1.064300e+10,8.536475e+08,0.07474,China Gas Holdings Limited operates as a gas o...,0.09136
337,Dominion Energy Inc,Utilities,Multi-Utilities,Multi-Utilities,D,0.0637,0.9889,15.522222,0.3,3.506916e+10,3.183400e+10,-3.939500e+09,0.08226,"Dominion Energy, Inc. produces and distributes...",0.45904
58,GAIL (India) Ltd,Utilities,Gas Utilities,Gas Utilities,GAIL.BO,0.0609,0.5477,15.469483,0.2,8.665982e+11,9.874700e+10,NaN,NaN,GAIL (India) Limited operates as a natural gas...,0.04005
512,Canadian Utilities Ltd,Utilities,Multi-Utilities,Multi-Utilities,CU.TO,0.0599,0.8463,13.985783,0.4,7.969825e+09,1.993500e+10,-3.940000e+08,0.09332,"Canadian Utilities Limited, together with its ...",0.45554
177,American Electric Power Co Inc,Utilities,Electric Utilities,Electric Utilities,AEP,0.0439,0.8516,19.705729,0.6,3.898337e+10,4.639700e+10,-4.120963e+09,0.08192,"American Electric Power Company, Inc., an elec...",0.34964
351,Fortis Inc,Utilities,Electric Utilities,Electric Utilities,FTS,0.0439,0.7534,18.371560,0.4,1.948032e+10,4.061700e+10,-1.650500e+09,0.07199,Fortis Inc. operates as an electric and gas ut...,0.40648
434,Southern Co,Utilities,Electric Utilities,Electric Utilities,SO,0.0415,0.9751,23.683275,0.1,7.257611e+10,2.812500e+10,-3.176750e+09,0.08739,"The Southern Company, through its subsidiaries...",0.36737
411,PG&E Corp,Utilities,Electric Utilities,Electric Utilities,PCG,0.0391,0.0,17.967033,0.4,4.089528e+10,1.152000e+10,-6.420625e+09,0.08446,"PG&E Corporation, through its subsidiary, Paci...",0.31222


In [ ]:
present_keys = equity_etfs.columns.tolist()
for i in equity_etfs.index:
    ticker = equity_etfs.loc[i, 'Ticker']
    try:
        info = yf.Ticker(ticker).info
    except:
        print(ticker)
        continue
    keys = info.keys()
    for j in keys:
        if j not in present_keys:
            present_keys.append(j)
        if info[j] == 'Infinity':
            info[j] = 0
        equity_etfs.at[i, j] = info[j]

VUG
IJH
IJR
IWM
VO
VYM
SCHD
QUAL
JEPI
DGRO
COWZ
MOAT
AVUV
VGT


In [102]:
for i in equity_etfs.sort_values(by='Expense Ratio', ascending=False).index:
    print(equity_etfs.loc[i, 'Ticker'], '-' ,equity_etfs.loc[i, 'Name\n'], '-', i)
    print(equity_etfs.loc[i, 'longBusinessSummary'])

GRID - FIRST TRST NASD CL EDG SGIIF - 79
The fund will normally invest at least 90% of its net assets (including investment borrowings) in the common stocks and depositary receipts that comprise the index. The index includes companies that are primarily engaged and involved in electric grid, electric meters and devices, networks, energy storage and management, and enabling software used by the smart grid infrastructure sector. The fund is non-diversified.
TAN - INVESCO SOLAR ETF - 77
The fund will generally invest at least 90% of its total assets in the securities that comprise the underlying index. Strictly in accordance with its guidelines and mandated procedures, MAC Indexing LLC, has contracted with S&P DJI Netherlands B.V. to calculate and administer the underlying index, which seeks to track the performance of companies in global solar energy businesses. The fund is non-diversified.
PBW - INVESCO WILDERHILL CLEAN ENE - 67
The fund generally will invest at least 90% of its total a

In [127]:
 equity_etfs.drop(63, inplace=True)

In [168]:
total_stock_data.loc[total_stock_data['GICS Sector'] == 'Real Estate']

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
27,Multiplan Empreendimentos Imobiliarios SA,MULT3.SA,B3 S.A.,Real Estate,Real Estate,Real Estate Management & Development,Real Estate Operating Companies,South America,BRL,0.00,...,55 21 3031 5402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Instone Real Estate Group O.N.,INS.F,Frankfurt Stock Exchange,Real Estate,Real Estate Management & Development,Real Estate Management & Development,Real Estate Development,Europe,EUR,5.27,...,49 201 45355 934,1.672445e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,Digital Realty Trust Inc,DLR,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,North America,USD,114.97,...,415 738 6501,1.672445e+09,15639849.0,15949698.0,1.693440e+09,1.695946e+09,0.0517,7.54,0.0611,NaN
344,Extra Space Storage Inc,EXR,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,North America,USD,108.52,...,801 365 4855,1.672445e+09,6779313.0,4931384.0,1.693440e+09,1.695946e+09,0.0321,5.54,0.0392,NaN
373,Iron Mountain Inc,IRM,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,North America,USD,58.98,...,NaN,1.672445e+09,12112183.0,12121951.0,1.693440e+09,1.695946e+09,0.0415,7.70,0.0498,NaN
413,Physicians Realty Trust,DOC,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Health Care REITs,North America,USD,11.23,...,NaN,1.672445e+09,10695289.0,10006205.0,1.693440e+09,1.695946e+09,0.0449,5.89,0.0656,NaN
420,Public Storage,PSA,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,North America,USD,250.01,...,818 291 1015,1.672445e+09,4367081.0,4223198.0,1.693440e+09,1.695946e+09,0.0248,5.93,0.0318,NaN
448,The GEO Group Inc,GEO,"New York Stock Exchange, Inc.",Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,North America,USD,8.41,...,NaN,1.672445e+09,15245775.0,16974112.0,1.693440e+09,1.695946e+09,0.1209,6.93,0.1745,NaN
488,China World Trade Center Co Ltd,600007.SS,Shanghai Stock Exchange,Real Estate,Real Estate,Real Estate Management & Development,Real Estate Operating Companies,Asia-Pacific,CNY,17.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,China Vanke Co Ltd,000002.SZ,Shenzhen Stock Exchange,Real Estate,Real Estate,Real Estate Management & Development,Real Estate Development,Asia-Pacific,CNY,11.65,...,86 75 5255 31696,1.640909e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# IEMG
# VWO
# SCHD = benchamrk, check comp, dividend stocks

In [4]:
equity_etfs

,Unnamed: 0,Ticker,Name\n,Fund Asset Class Focus\n,Fund Market Cap Focus\n,Strategy,Fund Industry Focus\n,Fund Geographical Focus\n,Maturity Band\n,Fund Rating Class Focus\n,...,longName,firstTradeDateEpochUtc,timeZoneFullName,timeZoneShortName,uuid,messageBoardId,gmtOffSetMilliseconds,trailingPegRatio,phone,forwardPE
0,6,VUG,VANGUARD GROWTH ETF,Equity,Large-cap,Growth,N.A.,United States,N.A.,N.A.,...,Vanguard Growth Index Fund,1075473000,America/New_York,EDT,3fb3c54b-ac72-3c28-8a84-61a7401ffd66,finmb_22939698,-14400000,NaN,800-523-1036,NaN
1,9,IJH,ISHARES CORE S&P MIDCAP ETF,Equity,Mid-cap,Blend,N.A.,United States,N.A.,N.A.,...,iShares Core S&P Mid-Cap ETF,959347800,America/New_York,EDT,6a5b52bc-75c6-339e-aba2-7ee9bd7a6919,finmb_8798152,-14400000,NaN,1-800-474-2737,NaN
2,10,IJR,ISHARES CORE S&P SMALL-CAP E,Equity,Small-cap,Blend,N.A.,United States,N.A.,N.A.,...,iShares Core S&P Small-Cap ETF,959347800,America/New_York,EDT,157aab75-8e0e-318d-b915-b2d3a479304a,finmb_8798175,-14400000,NaN,1-800-474-2737,NaN
3,11,IWM,ISHARES RUSSELL 2000 ETF,Equity,Small-cap,Blend,N.A.,United States,N.A.,N.A.,...,iShares Russell 2000 ETF,959347800,America/New_York,EDT,600ed58e-8501-353d-a7c7-902732abdc68,finmb_8797658,-14400000,NaN,1-800-474-2737,NaN
4,12,VO,VANGUARD MID-CAP ETF,Equity,Mid-cap,Blend,N.A.,United States,N.A.,N.A.,...,Vanguard Mid Cap Index Fund,1075473000,America/New_York,EDT,e984ea8b-e853-31e6-8331-3e8e1d7eb849,finmb_28122059,-14400000,NaN,800-523-1036,NaN
5,14,VYM,VANGUARD HIGH DVD YIELD ETF,Equity,Large-cap,Value,N.A.,United States,N.A.,N.A.,...,Vanguard High Dividend Yield Index Fund,1163687400,America/New_York,EDT,025a7501-0342-3f56-b3d4-85ae7d0c5565,finmb_30273256,-14400000,NaN,NaN,NaN
6,15,SCHD,SCHWAB US DVD EQUITY ETF,Equity,Large-cap,Value,N.A.,United States,N.A.,N.A.,...,Schwab U.S. Dividend Equity ETF,1319117400,America/New_York,EDT,51763995-7570-386d-806d-a25eca52c2b1,finmb_141947998,-14400000,NaN,1-800-435-4000,NaN
7,17,QUAL,ISHARES MSCI USA QUALITY FAC,Equity,Large-cap,Blend,N.A.,United States,N.A.,N.A.,...,iShares MSCI USA Quality Factor ETF,1374154200,America/New_York,EDT,4beb4298-a27d-31f6-9027-5b529f29e111,finmb_240359439,-14400000,NaN,1-800-474-2737,NaN
8,19,JEPI,JPMORGAN EQUITY PREMIUM INCO,Equity,Broad Market,Value,N.A.,United States,N.A.,N.A.,...,JPMorgan Equity Premium Income ETF,1590067800,America/New_York,EDT,7655f992-107f-3662-8cf3-00e4c29b2d0f,finmb_653863829,-14400000,NaN,(800) 480-4111,NaN
9,20,DGRO,ISHARES CORE DIVIDEND GROWTH,Equity,Broad Market,Blend,N.A.,United States,N.A.,N.A.,...,iShares Core Dividend Growth ETF,1402579800,America/New_York,EDT,b54f6875-c45a-31c9-a438-36dcf1efcf8f,finmb_263934819,-14400000,NaN,1-800-474-2737,NaN


In [153]:
total_stock_data.loc[total_stock_data['Ticker'] == 'AVGO']

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
185,Broadcom Inc,AVGO,Nasdaq,Information Technology,Semiconductors & Semiconductor Equipment,Semiconductors & Semiconductor Equipment,Semiconductors,North America,USD,857.96,...,NaN,1.672445e+09,6254827.0,5944819.0,1.693440e+09,1.695946e+09,0.0152,2.45,0.0195,NaN


In [169]:
total_stock_data.loc[total_stock_data['Ticker'] == 'KO']

,Company Name,Ticker,Exchange,GICS Sector,GICS Industry Group,GICS Industry,GICS Sub-Industry,Region,currency,open,...,fax,compensationAsOfEpochDate,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,dateShortInterest,sharesPercentSharesOut,shortRatio,shortPercentOfFloat,industrySymbol
326,Coca-Cola Co,KO,"New York Stock Exchange, Inc.",Consumer Staples,"Food, Beverage and Tobacco",Beverages,Soft Drinks,North America,USD,54.38,...,NaN,1.672445e+09,23040297.0,22869361.0,1.693440e+09,1.695946e+09,0.0053,1.55,0.0054,NaN


In [161]:
(existentDiv['trailingPE'].mean())

15.414650802941177